In [62]:
from sympy import *
import numpy as np
import pandas as pd
import plotly.express as px
def biseccion(f,x,a,b,tol):
    lam_f=lambdify(x,f);lam_f_prima=lambdify(x,f.diff(x))
    A: list=[];B: list=[];C: list=[];fC:list=[];bk_ax:list=[];deriv:list=[]
    while(abs(b-a)>=2*tol):
      A.append(a);B.append(b)
      c=np.float64((b+a)/2.0);C.append(c);fC.append(lam_f(c));deriv.append(lam_f_prima(c))
      bk_ax.append(abs(b-a))
      if(lam_f_prima(c)==0.0):
        break
      if(lam_f_prima(c)<np.float64(0.0)):
        b=c
      else:
        a=c
    return pd.DataFrame({'a_k' : A, 'b_k' : B, 'c_k': C, 'f(c)':fC,"f '(c)":deriv,'|bk_ak|':bk_ax})
def seccion_aurea(f,x,a,b,tol):
    lam_f=lambdify(x,f)
    #Definimos el radio r
    r=(-1+np.sqrt(5))/2
    #Inicializamos las listas donde guardaremos las columnas de la tabla
    x2i_1: list=[];x2i: list=[];fC1:list=[];fC2:list=[];bk_ax:list=[];tipo:list=[]
    #Inicializamos el contador de iteraciones
    i: int=0
    #Verificamos que aun no alcanzamos la tolerancia y no pasamos de 100 iteraciones
    while(abs(b-a)>=tol and i<=100):
      #Calculamos los dos puntos de esta iteración de sección aurea
      c1=b-r*(b-a);c2=a+r*(b-a)
      #Evaluamos la función en ambos puntos
      fC1.append(lam_f(c1));fC2.append(lam_f(c2))
      #Guardamos los puntos y la longitud del intervalo
      x2i.append(c2);x2i_1.append(c1)
      bk_ax.append(abs(b-a))
      #Efectuamos el criterio de comparación de seccion aurea y modificamos el intervalo
      if(lam_f(c1)<lam_f(c2)):
        a=c1;tipo.append('I')
      elif(lam_f(c1)>=lam_f(c2)):
        b=c2;tipo.append('II')
      i+=1      
    return pd.DataFrame({'x_2i-1': x2i_1, 'x2i' :x2i, 'f(x_2i-1)': fC1, 'f(x2i)':fC2,'|bk-ak|':bk_ax,'Tipos':tipo})
def newton_rhapson(f,x,x0,tol):
    fun=lambdify(x,f);deriv=lambdify(x,f.diff(x),'numpy');deriv2=lambdify(x,f.diff(x).diff(x),'numpy')
    X1: list=[]; X0: list=[]; X0_X1: list=[]; fx0: list=[]; fprimax0: list=[] ; f2primax0: list=[]
    while(True):
      x1=x0 - deriv(x0)/deriv2(x0)
      X0.append(x0);X1.append(x1);X0_X1.append(abs(x0-x1));fx0.append(fun(x0));fprimax0.append(deriv(x0));f2primax0.append(deriv2(x0))
      if(abs(x0-x1)<=tol):
        break
      x0 = x1 
    return pd.DataFrame({'x_i':X0,'x_1+1':X1,'f(x_i)':fx0,"f '(x_i)":fprimax0, "f ''(x_i)": f2primax0,'E = | x_i+1 - x_i |': X0_X1})
def secante(f,x,x0,x1,tol)->pd.DataFrame:
     X0: list=[];X1: list=[];X2: list=[];X2_X1: list=[];fprimax0: list=[]; fprimax1: list=[]
     fun=lambdify(x,f)
     fprima=lambdify(x,f.diff(x))
     while(abs(x0-x1)>=tol):
       X1.append(x1);X0.append(x0);fprimax0.append(fprima(x0));fprimax1.append(fprima(x1))
       x2 = x1 - ((x1-x0) /(fprima(x1)-fprima(x0)))*fprima(x1)
       X2_X1.append(abs(x2-x1));
       x0, x1 = x1, x2
       X2.append(x2)
     return pd.DataFrame({"x_i-1":X0,"x_i":X1,"x_i+1":X2,"f '(x_i-1)": fprimax0,"f '(x_i)":fprimax1, "E = | x_i+1 - x_i |":X2_X1})
def graficar(sym_func, variable, x_range):
    func = lambdify(variable, sym_func, "numpy")
    x_values = np.linspace(x_range[0], x_range[1], 500)
    y_values = func(x_values)
    fig = px.line(x=x_values, y=y_values)
    fig.show()

In [63]:
x=symbols('x')
f=1/2*x**2-sin(x)
secante(f,x,2,1.5,0.0001)


,x_i-1,x_i,x_i+1,f '(x_i-1),f '(x_i),E = | x_i+1 - x_i |
0,2.000000,1.500000,0.775871,2.416147,1.429263,0.724129
1,1.500000,0.775871,0.743002,1.429263,0.062060,0.032869
2,0.775871,0.743002,0.739116,0.062060,0.006560,0.003885
3,0.743002,0.739116,0.739085,0.006560,0.000052,0.000031


In [64]:
graficar(f,x,(0,1))

In [65]:
x=symbols('x')
f=4*x**3-3*x**4
newton_rhapson(f,x,0.7,0.25)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,0.700000,2.800000,0.651700,1.764000,-0.840000,2.100000
1,2.800000,2.012500,-96.588800,-169.344000,-215.040000,0.787500
2,2.012500,1.507817,-16.607539,-49.209398,-97.505625,0.504683
3,1.507817,1.204385,-1.794416,-13.854353,-45.658856,0.303432
4,1.204385,1.051791,0.675821,-3.557642,-23.314345,0.152595


In [66]:
x=symbols('x')
f=-x**2-1
seccion_aurea(f,x,-1,0.75,0.1)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,-0.331559,0.081559,-1.109932,-1.006652,1.750000,I
1,0.081559,0.336881,-1.006652,-1.113489,1.081559,II
2,-0.076238,0.081559,-1.005812,-1.006652,0.668441,II
3,-0.173762,-0.076238,-1.030193,-1.005812,0.413119,I
4,-0.076238,-0.015965,-1.005812,-1.000255,0.255322,I
5,-0.015965,0.021286,-1.000255,-1.000453,0.157797,II


In [67]:
f=12*x - 3*x**4-2*x**6
x=symbols('x')
biseccion(f,x,0,2,0.0001)

,a_k,b_k,c_k,f(c),f '(c),|bk_ak|
0,0.000000,2.000000,1.000000,7.000000,-12.000000,2.000000
1,0.000000,1.000000,0.500000,5.781250,10.125000,1.000000
2,0.500000,1.000000,0.750000,7.694824,4.089844,0.500000
3,0.750000,1.000000,0.875000,7.843864,-2.193970,0.250000
4,0.750000,0.875000,0.812500,7.867181,1.314365,0.125000
5,0.812500,0.875000,0.843750,7.882908,-0.339699,0.062500
6,0.812500,0.843750,0.828125,7.881504,0.511253,0.031250
7,0.828125,0.843750,0.835938,7.883868,0.091892,0.015625
8,0.835938,0.843750,0.839844,7.883810,-0.122357,0.007812
9,0.835938,0.839844,0.837891,7.883944,-0.014848,0.003906


Ejercicios

In [68]:
x=symbols('x')
f=x**4-5*x**3-2*x**2+24*x
biseccion(f,x,0,2,0.01)

,a_k,b_k,c_k,f(c),f '(c),|bk_ak|
0,0.000,2.00000,1.000000,18.000000,9.000000,2.00000
1,1.000,2.00000,1.500000,19.687500,-2.250000,1.00000
2,1.000,1.50000,1.250000,19.550781,3.375000,0.50000
3,1.250,1.50000,1.375000,19.795166,0.539062,0.25000
4,1.375,1.50000,1.437500,19.784927,-0.864258,0.12500
5,1.375,1.43750,1.406250,19.801011,-0.164429,0.06250
6,1.375,1.40625,1.390625,19.800837,0.186905,0.03125


In [69]:
graficar(f,x,(0,2))

In [70]:
f=x**2-6*x+1
biseccion(f,x,0,6,0.25)

,a_k,b_k,c_k,f(c),f '(c),|bk_ak|
0,0,6,3.0,-8.0,0.0,6


In [71]:
graficar(f,x,(0,6))

In [72]:
f=2*x**3-9*x**2+12*x
biseccion(f,x,0,1.5,0.15)

,a_k,b_k,c_k,f(c),f '(c),|bk_ak|
0,0.00,1.500,0.7500,4.781250,1.875000,1.500
1,0.75,1.500,1.1250,4.957031,-0.656250,0.750
2,0.75,1.125,0.9375,4.987793,0.398438,0.375


In [73]:
graficar(f,x,(0,1.5))

In [74]:
f=-x**2+2*x
seccion_aurea(f,x,-3,5,0.05)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,0.055728,1.944272,0.108351,0.108351,8.000000,II
1,-1.111456,0.055728,-3.458247,0.108351,4.944272,I
2,0.055728,0.777088,0.108351,0.950310,3.055728,I
3,0.777088,1.222912,0.950310,0.950310,1.888544,II
4,0.501553,0.777088,0.751550,0.950310,1.167184,I
5,0.777088,0.947378,0.950310,0.997231,0.721360,I
6,0.947378,1.052622,0.997231,0.997231,0.445825,II
7,0.882333,0.947378,0.986154,0.997231,0.275535,I
8,0.947378,0.987578,0.997231,0.999846,0.170290,I
9,0.987578,1.012422,0.999846,0.999846,0.105245,II


In [75]:
graficar(f,x,(-3,5))

In [76]:
f=x-exp(x)
seccion_aurea(f,x,-1,3,0.05)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,0.527864,1.472136,-1.167443,-2.886399,4.000000,II
1,-0.055728,0.527864,-1.001524,-1.167443,2.472136,II
2,-0.416408,-0.055728,-1.075819,-1.001524,1.527864,I
3,-0.055728,0.167184,-1.001524,-1.014788,0.944272,II
4,-0.193496,-0.055728,-1.017569,-1.001524,0.583592,I
5,-0.055728,0.029417,-1.001524,-1.000437,0.360680,I
6,0.029417,0.082039,-1.000437,-1.003459,0.222912,II
7,-0.003106,0.029417,-1.000005,-1.000437,0.137767,II
8,-0.023206,-0.003106,-1.000267,-1.000005,0.085145,I
9,-0.003106,0.009317,-1.000005,-1.000044,0.052622,II


In [77]:
graficar(f,x,(-1,3))

In [88]:
f=(x**2-0.01)*exp(-10*x)
seccion_aurea(f,x,-1,0,1*10**-4)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,-0.618034,-0.381966,179.717665,6.195416,1.000000,II
1,-0.763932,-0.618034,1192.114051,179.717665,0.618034,II
2,-0.854102,-0.763932,3684.194699,1192.114051,0.381966,II
3,-0.909830,-0.854102,7311.120014,3684.194699,0.236068,II
4,-0.944272,-0.909830,11122.866771,7311.120014,0.145898,II
5,-0.965558,-0.944272,14395.910915,11122.866771,0.090170,II
6,-0.978714,-0.965558,16875.378355,14395.910915,0.055728,II
7,-0.986844,-0.978714,18613.390239,16875.378355,0.034442,II
8,-0.991869,-0.986844,19774.519177,18613.390239,0.021286,II
9,-0.994975,-0.991869,20527.527167,19774.519177,0.013156,II


In [89]:
graficar(f,x,(0,5))

In [91]:
f=1/2*x**2-sin(x)
newton_rhapson(f,x,0.5,1*10**-5)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,0.500000,0.755222,-0.354426,-3.775826e-01,1.479426,2.552224e-01
1,0.755222,0.739142,-0.400270,2.710331e-02,1.685451,1.608075e-02
2,0.739142,0.739085,-0.400489,9.461538e-05,1.673654,5.653223e-05
3,0.739085,0.739085,-0.400489,1.180978e-09,1.673612,7.056461e-10


In [93]:
graficar(f,x,(0,1))

In [95]:
f=x**2-6*x+1
newton_rhapson(f,x,5,1*10**-5)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,5.0,3.0,-4.0,4.0,2,2.0
1,3.0,3.0,-8.0,0.0,2,0.0


In [96]:
graficar(f,x,(0,10))

In [97]:
f=2*x**3-9*x**2+12*x
newton_rhapson(f,x,.75,1*10**-5)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,0.750000,0.958333,4.781250,1.875000,-9.000000,0.208333
1,0.958333,0.998397,4.994647,0.260417,-6.500000,0.040064
2,0.998397,0.999997,4.999992,0.009631,-6.019231,0.001600
3,0.999997,1.000000,5.000000,0.000015,-6.000031,0.000003


In [98]:
graficar(f,x,(0,1.5))

In [107]:
f=x-exp(x)
newton_rhapson(f,x,-1,1*10**-5)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,-1.000000e+00,7.182818e-01,-1.367879,6.321206e-01,-0.367879,1.718282e+00
1,7.182818e-01,2.058711e-01,-1.332625,-1.050906e+00,-2.050906,5.124107e-01
2,2.058711e-01,1.980909e-02,-1.022724,-2.285949e-01,-1.228595,1.860620e-01
3,1.980909e-02,1.949109e-04,-1.000198,-2.000659e-02,-1.020007,1.961418e-02
4,1.949109e-04,1.899390e-08,-1.000000,-1.949299e-04,-1.000195,1.948919e-04
5,1.899390e-08,1.079323e-16,-1.000000,-1.899390e-08,-1.000000,1.899390e-08


In [108]:
graficar(f,x,(-2,2))

In [109]:
f=x**2+4*cos(x)
graficar(f,x,(1,2))

In [114]:
seccion_aurea(-f,x,1,2,0.2)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,1.381966,1.618034,-2.660671,-2.429154,1.000000,I
1,1.618034,1.763932,-2.429154,-2.343707,0.618034,I
2,1.763932,1.854102,-2.343707,-2.319570,0.381966,I
3,1.854102,1.909830,-2.319570,-2.317147,0.236068,I


In [115]:
newton_rhapson(f,x,1,0.2)

,x_i,x_1+1,f(x_i),f '(x_i),f ''(x_i),E = | x_i+1 - x_i |
0,1.000000,-7.472741,3.161209,-1.365884,-0.161209,8.472741
1,-7.472741,14.478521,57.330143,-11.232667,0.511709,21.951262
2,14.478521,6.935115,208.288517,25.187833,3.339053,7.543406
3,6.935115,16.635684,51.275483,11.443344,-1.179657,9.700569
4,16.635684,8.343938,274.347348,36.472389,4.398638,8.291747
5,8.343938,4.954633,67.738946,13.158461,3.882348,3.389305
6,4.954633,-8.301318,25.507911,13.792474,1.040474,13.255951
7,-8.301318,-4.817320,67.181618,-12.996226,3.730262,3.483998
8,-4.817320,3.792574,23.625525,-13.612639,1.581046,8.609894
9,3.792574,1.861061,11.201664,10.009020,5.181957,1.931514


In [117]:
f=8*exp(1-x)+7*ln(x)
graficar(f,x,(1,2))

In [119]:
seccion_aurea(-f,x,1,2,0.23)

,x_2i-1,x2i,f(x_2i-1),f(x2i),|bk-ak|,Tipos
0,1.381966,1.618034,-7.724696,-7.680507,1.000000,I
1,1.618034,1.763932,-7.680507,-7.699467,0.618034,II
2,1.527864,1.618034,-7.686003,-7.680507,0.381966,I
3,1.618034,1.673762,-7.680507,-7.683814,0.236068,II


In [120]:
secante(f,x,1,2,0.23)

,x_i-1,x_i,x_i+1,f '(x_i-1),f '(x_i),E = | x_i+1 - x_i |
0,1,2.000000,1.642275,-1.000000,0.556964,0.357725
1,2,1.642275,1.604162,0.556964,0.053627,0.038113


In [121]:
8*exp(1-1.604162)+7*ln(1.604162)

7.68046834336696